## Feature Engineering from sessions.csv

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix

from os import makedirs
from os.path import exists, join

%matplotlib inline

In [2]:
intermediate_path = './intermediate/'

In [3]:
df_s = pd.read_csv('./input/sessions.csv')

In [4]:
df_s.shape

(10567737, 6)

In [5]:
# just checking a random user
#df_s[df_s.user_id == 'awen0gt0iw']

In [6]:
# eliminate user_id with NaN value
print df_s[pd.isnull(df_s.user_id)].shape

df_s = df_s[pd.isnull(df_s.user_id) == False]

(34496, 6)


In [7]:
# fill NaN for other columns

df_s['action'].fillna('Nada', inplace=True)
df_s['action_detail'].fillna('Nada', inplace=True)
df_s['action_type'].fillna('Nada', inplace=True)

df_s['secs_elapsed'].fillna(0, inplace=True)

In [8]:
# just a check on values that allude to translations
if False:
    df_s = pd.read_csv('./input/sessions.csv')
    l = df_s.action.value_counts().index
    trans = [s for s in l if 'transl' in s ]
    print trans
    l = df_s.action_detail.value_counts().index
    print [s for s in l if 'transl' in s ]
    print df_s[df_s.action.isin(trans)].count()

In [9]:
df_s.shape

(10533241, 6)

In [10]:
df_s.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,Nada,Nada,Windows Desktop,319
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753
2,d1mm9tcy42,lookup,Nada,Nada,Windows Desktop,301
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141
4,d1mm9tcy42,lookup,Nada,Nada,Windows Desktop,435


In [11]:
# remove hiphens from action columns.
# '-' are considered separators in CountVectorizer

for c in ['action', 'action_type', 'action_detail']:
    l = df_s[c].unique()
    l = [s for s in l if not pd.isnull(s)]
    print [s for s in l if '-' in s]
    df_s[c] = [ s.replace('-', '') for s in df_s[c]]
    l = df_s[c].unique()
    l = [s for s in l if not pd.isnull(s)]
    print [s for s in l if '-' in s]

['southern-europe', 'social-media', 'united-states', 'south-america', 'rest-of-world']
[]
['-unknown-']
[]
['-unknown-']
[]


In [12]:
# remove spaces from device_type

df_s['device_type'] = [ s.replace(' ', "_") for s in df_s.device_type]

In [14]:
# adding spaces for CountVectorizer split

df_s['device_type']   = ["%s "%w for w in df_s['device_type']]

df_s['action_type']   = ["%s "%w for w in df_s['action_type']]

df_s['action']        = ["%s "%w for w in df_s['action']]
df_s['action_detail'] = ["%s "%w for w in df_s['action_detail']]

In [15]:
# converting secs_elapsed to int 
df_s['secs_elapsed'] = df_s.secs_elapsed.astype(int)

In [16]:
# sessionize action sequence, create session_id column

nw_se_th = 30   # nw_se_th : new session threshold

def generate_session_id(df, th):  

    s_id = 1
    s_ve = []
    
    for se in df['secs_elapsed']:
        if se < th * 60:
            s_ve.append(s_id)
        else:
            s_ve.append(s_id)
            s_id += 1
    df['session_id' + '_' + str(th)] = s_ve
    return df

In [17]:
df_s.secs_elapsed.describe()

count    10533241.000000
mean        19161.568128
std         88362.631674
min             0.000000
25%           208.000000
50%          1108.000000
75%          8191.000000
max       1799977.000000
Name: secs_elapsed, dtype: float64

In [18]:
# 'sessionizing' user navigation
df_s = df_s.groupby('user_id').apply(lambda x: generate_session_id(x, 30))

In [19]:
# checking a random user
df_s[df_s.user_id == 'd1mm9tcy42']
#df_s[df_s.user_id == 'awen0gt0iw']

,user_id,action,action_type,action_detail,device_type,secs_elapsed,session_id_30
0,d1mm9tcy42,lookup,Nada,Nada,Windows_Desktop,319,1
1,d1mm9tcy42,search_results,click,view_search_results,Windows_Desktop,67753,1
2,d1mm9tcy42,lookup,Nada,Nada,Windows_Desktop,301,2
3,d1mm9tcy42,search_results,click,view_search_results,Windows_Desktop,22141,2
4,d1mm9tcy42,lookup,Nada,Nada,Windows_Desktop,435,3
5,d1mm9tcy42,search_results,click,view_search_results,Windows_Desktop,7703,3
6,d1mm9tcy42,lookup,Nada,Nada,Windows_Desktop,115,4
7,d1mm9tcy42,personalize,data,wishlist_content_update,Windows_Desktop,831,4
8,d1mm9tcy42,index,view,view_search_results,Windows_Desktop,20842,4
9,d1mm9tcy42,lookup,Nada,Nada,Windows_Desktop,683,5


In [22]:
# grouping by user_id
# actions and device_type as sum of strings values - concatenation of actions separated by spaces
# secs_elapsed as sum of secs_elapsed int values - total cumulative seconds 
# session_id max- total number of sessions per user

grouped = df_s.groupby('user_id', sort = False)[['action', 'action_type', 'action_detail']]

df_s_a = grouped.sum()

df_s_se = df_s.groupby('user_id', sort = False)['secs_elapsed'].agg({'sum': np.sum})

# selecting columns with session_id counts
session_id_cols = [c for c in df_s.columns if 'session_id' in c]
df_s_sc = df_s.groupby('user_id', sort = False)[session_id_cols].max()

# creating device type datagram
df_s_dt = df_s.groupby('user_id', sort = False)['device_type'].sum()

In [23]:
df_s_se.head()

,sum
user_id,
d1mm9tcy42,3427529
yo8nz8bqcq,207842
4grx6yxeby,1135444
ncf87guaf0,3755100
4rvqpxoh3h,2555


In [24]:
print df_s_a.shape

print df_s_se.shape

print df_s_sc.shape

print df_s_dt.shape

(135483, 3)
(135483, 1)
(135483, 1)
(135483,)


In [25]:
df_s_a.head()

,action,action_type,action_detail
user_id,,,
d1mm9tcy42,lookup search_results lookup search_results lo...,Nada click Nada click Nada click Nada data vie...,Nada view_search_results Nada view_search_resu...
yo8nz8bqcq,dashboard create confirm_email show show_perso...,view submit click view data view data data Nada,dashboard create_user confirm_email_link p3 us...
4grx6yxeby,verify create Nada pending requested header_us...,unknown submit message_post booking_request vi...,unknown create_user message_post pending p5 he...
ncf87guaf0,lookup show search_results search_results show...,Nada view click click view view click data dat...,Nada p3 view_search_results view_search_result...
4rvqpxoh3h,campaigns active create notifications listings...,unknown unknown unknown unknown unknown unknow...,unknown unknown unknown unknown unknown unknow...


In [26]:
df_s_a.shape

(135483, 3)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vec_dt = CountVectorizer(binary=True)

vec_aa = CountVectorizer(ngram_range=(1, 2))
vec_at = CountVectorizer(ngram_range=(1, 2))
vec_ad = CountVectorizer(ngram_range=(1, 2))


In [28]:
df_s_a.shape

(135483, 3)

In [29]:
# need labels to select train data
df_train = pd.read_csv('./input/train_users_2.csv', index_col=0)

# ids in session file corresponding to train
session_train_id = df_s_a.index.intersection(df_train.index)

In [30]:
session_train_id.shape

(73815,)

In [31]:
df_s_a.shape

(135483, 3)

In [32]:
# vectorize action, action_type, action_detail all unigrams, all bigrams

vec_dt    = vec_dt.fit(df_s_dt.ix[session_train_id].values)
vec_aa    = vec_aa.fit(df_s_a.ix[session_train_id].action.values)
vec_at    = vec_at.fit(df_s_a.ix[session_train_id].action_type.values)
vec_ad    = vec_ad.fit(df_s_a.ix[session_train_id].action_detail.values)

In [33]:
#session_all_aa_ad = vec_aa_ad.transform(df_s_a.aa_ad.values)

session_all_dt    = vec_dt.transform(df_s_dt.values)
session_all_aa    = vec_aa.transform(df_s_a.action.values)
session_all_at    = vec_at.transform(df_s_a.action_type.values)
session_all_ad    = vec_ad.transform(df_s_a.action_detail.values)

In [34]:
# checking vectorized matrices shapes
print session_all_dt.shape
print session_all_aa.shape
print session_all_at.shape
print session_all_ad.shape

(135483, 14)
(135483, 17426)
(135483, 95)
(135483, 6673)


In [35]:
# concatenating (action, action_detail) with action_type 

#session_all_a = hstack([session_all_aa_ad, session_all_at])
session_all_a = hstack([session_all_at, session_all_aa, session_all_ad])

In [37]:
session_all_a = session_all_a.tocsr()

#### Building session_all

In [38]:
# add column 'unique_devices' number of unique devices
session_all = pd.DataFrame(session_all_dt.sum(axis = 1), index=df_s_se.index, columns=['unique_devices'])

# add columns for vectorized device types 
#session_all = pd.DataFrame(session_all_dt.toarray(), index=df_s_a.index, columns=vec_dt.get_feature_names())

In [39]:
# add column 'secs elapsed' cumulative seconds elapsed
session_all = pd.concat([session_all, df_s_se], axis = 1)
print session_all.shape

(135483, 2)


In [40]:
# add column 'session_id' represents total number of sessions
session_all = pd.concat([session_all, df_s_sc], axis = 1)
#session_all.rename(columns={'session_id' : 'session_count'}, inplace=True)

In [41]:
session_all.head()

,unique_devices,sum,session_id_30
user_id,,,
d1mm9tcy42,2,3427529,48
yo8nz8bqcq,1,207842,7
4grx6yxeby,2,1135444,10
ncf87guaf0,3,3755100,72
4rvqpxoh3h,1,2555,1


In [45]:
# Univariate feature selection for action uni and bigrams

# Creating and Encoding y
from sklearn.preprocessing import LabelEncoder
labels = df_train.ix[session_train_id].country_destination
le = LabelEncoder()
y = le.fit_transform(labels.values) 

from sklearn.feature_selection import SelectKBest, chi2

X = session_all_a[:session_train_id.shape[0]].copy()
sel = SelectKBest(chi2, k=1000)
sel.fit(X,y)

X_new = sel.transform(session_all_a)

X_new = pd.DataFrame(X_new.toarray(), index=df_s_a.index)

In [46]:
# adding action uni and bigram features after feature selection

session_all = pd.concat([session_all, X_new], axis = 1)

In [47]:
session_all.shape

(135483, 1003)

In [48]:
session_all.head()

,unique_devices,sum,session_id_30,0,1,2,3,4,5,6,...,990,991,992,993,994,995,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,
d1mm9tcy42,2,3427529,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yo8nz8bqcq,1,207842,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4grx6yxeby,2,1135444,10,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ncf87guaf0,3,3755100,72,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4rvqpxoh3h,1,2555,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
if not exists(intermediate_path):
    makedirs(intermediate_path)

session_all.to_csv( join(intermediate_path, 'session_features_1000.csv') )